In [19]:
%reload_ext autoreload
%autoreload 2

In [20]:
import sys
import os

# Add src directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

from pricing.triple_win import TripleWinPricing, TrackingTripleWinPricing
from pricing.buyer import BuyerBlock
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt


In [21]:
df_shapley = pd.read_csv("../tables/unified_shapley_matrix_10sellers.csv")

selected_columns = []

MODELS_NUM = 10

for i in range(10):
    selected_columns.append(f"seller_{i}")
SV = df_shapley[selected_columns].sample(MODELS_NUM).values.transpose()
SV = SV / SV.sum(axis=0, keepdims=True)

In [23]:
# %% [markdown]
# # Verifying the platform-commission bound τ* with a uniform ad–valorem fee
# 
# - Market primitives per model j: κ_Mj, δ_j, S_j = Σ_i κ_Di, ρ_j = Σ_k ω_jk, and Rmin_j = min_k R_{B_k→M_j}
# - Buyer prices with uniform grossing α = 1/(1-τ):  pMj(α) = [α κ_Mj + α^2 (1+δ_j) S_j] / (1 - α ρ_j)
# - Feasibility ⇔ pMj(α) ≤ Rmin_j for all j  and  α ρ_j < 1 for all j
# - The maximal feasible α* is the minimum across models of the positive root of
#     (1+δ_j) S_j α^2 + (κ_Mj + ρ_j Rmin_j) α - Rmin_j = 0
#   (with the linear special case when S_j = 0).
# - τ* = 1 - 1/α*; if α* ≤ 1, the maximal nonnegative fee is τ* = 0.

import numpy as np

rng = np.random.default_rng(42)

def build_random_market(J=10, seed=None):
    """Return dict of arrays: kappa_M, delta, S, rho, Rmin with rho<1."""
    Rmin = rng.uniform(10.0, 30.0, size=J)
    kM   = rng.uniform(0.5,  5.0,  size=J)
    delta= rng.uniform(0.0,  0.5,  size=J)
    S    = rng.uniform(0.0,  1.5,  size=J)   # sum of κ_Di per model
    rho  = rng.uniform(0.2,  0.95, size=J)   # strictly < 1
    return dict(kappa_M=kM, delta=delta, S=S, rho=rho, Rmin=Rmin)

def pMj_of_alpha(alpha, market):
    kM, delta, S, rho = (market[k] for k in ["kappa_M","delta","S","rho"])
    denom = 1.0 - alpha * rho
    if np.any(denom <= 0):
        # Return +inf where denominator is nonpositive
        out = np.full_like(kM, np.inf, dtype=float)
        mask = denom > 0
        out[mask] = (alpha*kM[mask] + (alpha**2)*(1.0+delta[mask])*S[mask]) / denom[mask]
        return out
    return (alpha*kM + (alpha**2)*(1.0+delta)*S) / denom

def alpha_star_closed_form(market):
    """Per model positive root, then take min across j, respecting α < 1/ρ."""
    kM, delta, S, rho, Rmin = (market[k] for k in ["kappa_M","delta","S","rho","Rmin"])
    alpha_caps = np.empty_like(kM, dtype=float)
    for j in range(kM.size):
        a = (1.0 + delta[j]) * S[j]
        b = kM[j] + rho[j] * Rmin[j]
        c = - Rmin[j]
        if a > 0:
            disc = b*b - 4.0*a*c
            alpha_pos = (-b + np.sqrt(disc)) / (2.0*a)
        else:
            # linear case: a=0 ⇒ b α + c ≤ 0 ⇒ α ≤ -c/b = Rmin / (kM + ρ Rmin)
            alpha_pos = Rmin[j] / max(b, 1e-16)
        # domain guard α ρ < 1
        if rho[j] > 0:
            alpha_caps[j] = min(alpha_pos, (1.0 - 1e-12) / rho[j])
        else:
            alpha_caps[j] = alpha_pos
    alpha_star = float(np.min(alpha_caps))
    return alpha_star, alpha_caps

def phi(alpha, market):
    """φ(α) = min_j Rmin_j (1 - α ρ_j) / (κ_Mj + α (1+δ_j) S_j)."""
    kM, delta, S, rho, Rmin = (market[k] for k in ["kappa_M","delta","S","rho","Rmin"])
    denom = kM + alpha * (1.0 + delta) * S
    denom = np.maximum(denom, 1e-16)
    gj = Rmin * np.maximum(0.0, 1.0 - alpha * rho) / denom
    return float(np.min(gj))

def alpha_star_bisection(market, lo=0.0, hi=None, tol=1e-10, max_iter=200):
    """Solve φ(α) - α = 0 by bisection on [lo, hi), with hi < min_j 1/ρ_j."""
    rho = market["rho"]
    hi_cap = np.min(np.where(rho > 0, 1.0 / rho, np.inf)) - 1e-12
    if hi is None or not np.isfinite(hi) or hi > hi_cap:
        hi = hi_cap
    # Ensure bracket signs: h(lo) = φ(lo) - lo >= 0; h(hi) <= 0
    def h(a): return phi(a, market) - a
    hlo, hhi = h(lo), h(hi)
    if hlo < 0:
        return lo  # no positive headroom; α*≈lo (we'll clamp to ≥1 later)
    if hhi > 0:
        # φ(α) - α stays positive on [lo, hi] → push hi up to cap; if still >0, return hi
        return hi
    for _ in range(max_iter):
        mid = 0.5*(lo + hi)
        hmid = h(mid)
        if hmid >= 0:
            lo = mid
        else:
            hi = mid
        if hi - lo <= tol * max(1.0, lo):
            break
    return 0.5*(lo + hi)

def alpha_star_damped_fp(market, alpha0=None, eta=0.5, tol=1e-10, max_iter=10000):
    """Optional damped fixed-point: α_{t+1} = (1-η) α_t + η φ(α_t)."""
    rho = market["rho"]
    hi_cap = np.min(np.where(rho > 0, 1.0 / rho, np.inf)) - 1e-12
    alpha = 1.0 if alpha0 is None else float(alpha0)
    alpha = min(max(alpha, 0.0), hi_cap)
    for _ in range(max_iter):
        nxt = (1.0 - eta) * alpha + eta * phi(alpha, market)
        nxt = min(max(nxt, 0.0), hi_cap)
        if abs(nxt - alpha) <= tol * max(1.0, alpha):
            return nxt
        alpha = nxt
    return alpha

def verify_one_market(market, eps=1e-6, quiet=False):
    a_closed, caps = alpha_star_closed_form(market)
    a_fp = alpha_star_bisection(market, lo=0.0)
    a_damped = alpha_star_damped_fp(market, alpha0=a_fp, eta=0.5)
    # choose α* from fixed-point (equivalent to closed form)
    a_star = a_fp
    tau_star = max(0.0, 1.0 - 1.0 / max(a_star, 1.0))  # clamp τ ≥ 0

    p_at = pMj_of_alpha(a_star, market)
    feasible_at = np.all(p_at <= market["Rmin"] + 1e-8)
    binds = np.any(np.isclose(p_at, market["Rmin"], rtol=1e-8, atol=1e-8))

    a_plus = min(a_star + eps, np.min(np.where(market["rho"] > 0, 1.0 / market["rho"], np.inf)) - 1e-12)
    p_above = pMj_of_alpha(a_plus, market)
    violate_above = np.any(p_above > market["Rmin"] + 1e-6)

    if not quiet:
        print("=== Commission bound verification (one market) ===")
        print(f"α*_closed = {a_closed:.10f},   α*_fp = {a_fp:.10f},   α*_damped = {a_damped:.10f}")
        print(f"τ* = {tau_star:.6f}")
        print(f"Feasible at α*: {'YES' if feasible_at else 'NO'};  Binding at α*: {'YES' if binds else 'NO'}")
        if a_star > 1.0:
            print(f"Violation just above α*: {'YES' if violate_above else 'NO'}")
        else:
            print("No positive headroom (α* ≤ 1): τ* = 0; binding/violation at α*>1 not applicable.")
        # Show first few models
        Rmin = market["Rmin"]
        for j in range(min(10, Rmin.size)):
            slack = Rmin[j] - p_at[j]
            print(f"  j={j:02d}: Rmin={Rmin[j]:.6f},  pMj(α*)={p_at[j]:.6f},  slack={slack:+.6e}")
        print("=================================================")
    return dict(alpha_star=a_star, tau_star=tau_star, feasible_at=feasible_at,
                binds=binds, violate_above=violate_above,
                alpha_closed=a_closed, alpha_fp=a_fp, alpha_damped=a_damped)

# ---- Try one market
mkt = build_random_market(J=SV.shape[1])
_ = verify_one_market(mkt)

# ---- Batch check
def batch(n=20, J=12, seed=123):
    rng = np.random.default_rng(seed)
    stats = dict(match=0, feas=0, bind=0, violate=0, no_headroom=0)
    for t in range(n):
        m = build_random_market(J=J)
        out = verify_one_market(m, quiet=True)
        if abs(out["alpha_closed"] - out["alpha_fp"]) <= 1e-8 * max(1.0, out["alpha_fp"]):
            stats["match"] += 1
        stats["feas"] += int(out["feasible_at"])
        stats["bind"] += int(out["binds"])
        stats["violate"] += int(out["violate_above"])
        if out["alpha_star"] <= 1.0:
            stats["no_headroom"] += 1
    print("\n=== Batch summary ===")
    print(f"Markets: {n}")
    print(f"closed-form ≈ fixed-point: {stats['match']}/{n}")
    print(f"Feasible at α*:           {stats['feas']}/{n}")
    print(f"Binding at α*:            {stats['bind']}/{n}")
    print(f"Violation just above α*:  {stats['violate']}/{n}")
    print(f"No positive headroom (α*≤1): {stats['no_headroom']}/{n}")
    print("=======================")

batch(n=20, J=12)


=== Commission bound verification (one market) ===
α*_closed = 0.8679038097,   α*_fp = 0.8679038096,   α*_damped = 0.8679038097
τ* = 0.000000
Feasible at α*: YES;  Binding at α*: YES
No positive headroom (α* ≤ 1): τ* = 0; binding/violation at α*>1 not applicable.
  j=00: Rmin=25.479121,  pMj(α*)=5.615032,  slack=+1.986409e+01
  j=01: Rmin=18.777569,  pMj(α*)=18.777569,  slack=+1.315165e-09
  j=02: Rmin=27.171958,  pMj(α*)=9.431898,  slack=+1.774006e+01
  j=03: Rmin=23.947361,  pMj(α*)=6.825326,  slack=+1.712203e+01
  j=04: Rmin=11.883547,  pMj(α*)=10.196651,  slack=+1.686896e+00
  j=05: Rmin=29.512447,  pMj(α*)=5.143715,  slack=+2.436873e+01
  j=06: Rmin=25.222794,  pMj(α*)=4.843223,  slack=+2.037957e+01
  j=07: Rmin=25.721286,  pMj(α*)=1.929520,  slack=+2.379177e+01
  j=08: Rmin=12.562273,  pMj(α*)=10.316205,  slack=+2.246068e+00
  j=09: Rmin=19.007719,  pMj(α*)=5.324958,  slack=+1.368276e+01

=== Batch summary ===
Markets: 20
closed-form ≈ fixed-point: 20/20
Feasible at α*:          

In [24]:
# Self-contained notebook cell: compute α* (commission frontier) via two methods,
# print all per-model prices p_{M_j}(α*), buyer reserves R_{jk}, and R_j^min;
# and verify that a tiny increase above α* violates at least one buyer reserve.
#
# Replace the MARKET SETUP block with your arrays:
#   kappa_M: shape (J,)         # κ_{M_j}
#   S      : shape (J,)         # S_j = sum_i κ_{D_i} for model j
#   delta  : shape (J,)         # δ_{M_j} ≥ 0
#   rho    : shape (J,)         # ρ_j ∈ [0,1)
#   R_list : list of 1-D arrays # for model j, reserves R_{B_k→M_j} for all buyers k

import numpy as np

# -------------------- MARKET SETUP (demo) --------------------
rng = np.random.default_rng(42)
J = 6                                      # number of models
kappa_M = rng.uniform(1.0, 5.0, size=J)    # κ_{M_j}
S       = rng.uniform(0.3, 1.5, size=J)    # S_j = sum_i κ_{D_i} (per model)
delta   = rng.uniform(0.0, 0.3, size=J)    # δ_{M_j} ≥ 0
rho     = rng.uniform(0.2, 0.8, size=J)    # ρ_j ∈ [0,1)

# Buyer reserves per model (heterogeneous K_j)
R_list = []
for j in range(J):
    Kj = rng.integers(2, 6)                # 2–5 buyers per model
    Rj = rng.uniform(8.0, 25.0, size=Kj)   # reserves (replace with your values)
    R_list.append(np.sort(Rj))             # sort for pretty printing

Rmin = np.array([Rj.min() for Rj in R_list], dtype=float)  # R_j^min

# -------------------- Helpers --------------------
def pM_closed(alpha, kappa_M, S, delta, rho):
    """Vector p_{M_j}(α) for all j (closed-form), valid if α ρ_j < 1."""
    denom = 1.0 - alpha * rho
    if np.any(denom <= 0):
        raise ValueError("alpha is too large: α ρ_j >= 1 for some j")
    return (alpha * kappa_M + (alpha**2) * (1.0 + delta) * S) / denom

def phi(alpha, kappa_M, S, delta, rho, Rmin):
    """φ(α) = min_j Rmin_j (1 - α ρ_j) / (κ_Mj + α (1+δ_j) S_j)."""
    denom = kappa_M + alpha * (1.0 + delta) * S
    denom = np.maximum(denom, 1e-14)
    vals = Rmin * (1.0 - alpha * rho) / denom
    return float(np.min(vals))

def alpha_fp_bisect(kappa_M, S, delta, rho, Rmin, tol=1e-12, max_iter=200):
    """Bisection on F(α) = φ(α) - α (fixed-point equation)."""
    rho_max = float(np.max(rho))
    alpha_hi = 0.999 / rho_max if rho_max > 0 else 10.0
    def F(a):  # fixed-point residual
        return phi(a, kappa_M, S, delta, rho, Rmin) - a
    lo, hi = 0.0, alpha_hi
    Flo, Fhi = F(lo), F(hi)
    if Fhi > 0:
        return float(hi), alpha_hi   # feasible throughout; α* = hi
    if Flo < 0:
        return 0.0, alpha_hi         # infeasible even at α=0
    for _ in range(max_iter):
        mid = 0.5 * (lo + hi)
        Fmid = F(mid)
        if Fmid >= 0:
            lo = mid
        else:
            hi = mid
        if hi - lo <= tol * max(1.0, lo):
            break
    return float(lo), alpha_hi

def alpha_closed_bisect(kappa_M, S, delta, rho, Rmin, alpha_hi, tol=1e-12, max_iter=200):
    """Bisection on h(α) = max_j ( pM_closed_j(α) - Rmin_j ), find largest α with h(α) ≤ 0."""
    def h(a):
        pm = pM_closed(a, kappa_M, S, delta, rho)
        return float(np.max(pm - Rmin))
    lo = 0.0
    hi = min(alpha_hi, 0.999 * alpha_hi)
    if h(hi) < 0:   # entire interval feasible
        return float(hi)
    if h(lo) > 0:   # infeasible even at 0
        return 0.0
    for _ in range(max_iter):
        mid = 0.5 * (lo + hi)
        hmid = h(mid)
        if hmid <= 0:
            lo = mid
        else:
            hi = mid
        if hi - lo <= tol * max(1.0, lo):
            break
    return float(lo)

# -------------------- Solve α via both methods --------------------
alpha_fp, alpha_hi = alpha_fp_bisect(kappa_M, S, delta, rho, Rmin, tol=1e-12)
alpha_closed = alpha_closed_bisect(kappa_M, S, delta, rho, Rmin, alpha_hi, tol=1e-12)

tau_fp     = 1.0 - 1.0 / max(alpha_fp, 1e-14) if alpha_fp > 0 else 0.0
tau_closed = 1.0 - 1.0 / max(alpha_closed, 1e-14) if alpha_closed > 0 else 0.0

print("=== Commission frontier (bisection methods) ===")
print(f"alpha_fp     = {alpha_fp:.12f}   ->   tau_fp     = {tau_fp:.6f}")
print(f"alpha_closed = {alpha_closed:.12f}   ->   tau_closed = {tau_closed:.6f}")
print(f"|alpha_fp - alpha_closed| = {abs(alpha_fp - alpha_closed):.3e}")
print("------------------------------------------------")

alpha_star = alpha_fp
pM_star = pM_closed(alpha_star, kappa_M, S, delta, rho)

# Print per-model values at α*
print("\nPer-model prices and reserves at α* (alpha_fp):")
for j in range(J):
    Rj = R_list[j]
    Rmin_j = Rmin[j]
    print(f"  j={j:02d}: pM*(j)={pM_star[j]:10.6f} | Rmin={Rmin_j:10.6f} | R_j={np.array2string(Rj, precision=6)}")

# Violation check just above α*
eps = 1e-6 * max(1.0, alpha_star)
pM_above = pM_closed(min(alpha_star + eps, 0.999 * alpha_hi), kappa_M, S, delta, rho)
violations = pM_above > Rmin + 1e-12
print("\nJust-above-α* feasibility check (α* + ε):")
print(f"  any violation? {'YES' if np.any(violations) else 'NO'}")
print(f"  violating models (indices): {np.where(violations)[0].tolist()}")


=== Commission frontier (bisection methods) ===
alpha_fp     = 0.773395427658   ->   tau_fp     = -0.293000
alpha_closed = 0.773395427658   ->   tau_closed = -0.293000
|alpha_fp - alpha_closed| = 3.431e-14
------------------------------------------------

Per-model prices and reserves at α* (alpha_fp):
  j=00: pM*(j)=  8.744664 | Rmin=  8.744664 | R_j=[ 8.744664 10.622921 11.308858 15.934257]
  j=01: pM*(j)=  5.538361 | Rmin= 13.539031 | R_j=[13.539031 14.297815 19.611832 20.660957 24.447665]
  j=02: pM*(j)=  7.571907 | Rmin= 10.208666 | R_j=[10.208666 11.221013]
  j=03: pM*(j)=  5.093577 | Rmin= 11.857459 | R_j=[11.857459 16.086984 19.386838]
  j=04: pM*(j)=  4.012514 | Rmin= 13.310233 | R_j=[13.310233 19.904507 21.680994 22.148417 22.155529]
  j=05: pM*(j)= 10.797402 | Rmin= 12.901578 | R_j=[12.901578 14.587132 19.602424]

Just-above-α* feasibility check (α* + ε):
  any violation? YES
  violating models (indices): [0]
